## Импорты

In [4]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score

from sklearn.preprocessing import normalize
from sklearn.preprocessing import StandardScaler 

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC,LinearSVC
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from catboost import CatBoostClassifier



PATH_TO_DATA="/Users/roman/finunichamp2018/2/data/"
PATH_TO_SAMPLE_SUBMIT="/Users/roman/finunichamp2018/2/data/Исходные/insclass_sample.csv"
PATH_TO_SUBMIT="/Users/roman/finunichamp2018/2/submits/"

sep=151405

In [5]:
def submit(y_pred,name):
    sub=pd.read_csv(PATH_TO_SAMPLE_SUBMIT,index_col="id")
    sub["target"]=y_pred
    sub.to_csv(PATH_TO_SUBMIT+name+".csv")

In [6]:
def tree_final(algo,name):
    algo.fit(tree_train.drop("target",axis=1),tree_train["target"])
    y_pred=algo.predict_proba(tree_test)[:,1]
    submit(y_pred,name)

In [7]:
def lin_final(algo,name):
    algo.fit(linear_train.drop("target",axis=1),linear_train["target"])
    y_pred=algo.predict_proba(tree_test)[:,1]
    submit(y_pred,name)

In [8]:
def cat_final(algo,name,cat_features):
    algo.fit(cat_train.drop("target",axis=1),cat_train["target"],cat_features=cat_features)
    y_pred=algo.predict_proba(cat_test)[:,1]
    submit(y_pred,name)

In [9]:
def xgb_final(algo,name):
    algo.fit(xgb_train.drop("target",axis=1),xgb_train["target"])
    y_pred=algo.predict_proba(xgb_test)[:,1]
    submit(y_pred,name)

_____

In [10]:
linear=pd.read_csv(PATH_TO_DATA+"cleared_linear.csv")
tree=pd.read_csv(PATH_TO_DATA+"cleared_tree.csv")
xgb_data=pd.read_csv(PATH_TO_DATA+"cleared_xgb.csv")
catBoost_data=pd.read_csv(PATH_TO_DATA+"cleared_cat.csv")


target=pd.read_csv(PATH_TO_DATA+"target.csv")

In [11]:
#Для линейных моделей
linear_train=linear[:sep]
linear_test=linear[sep:]

linear_train=pd.concat([linear_train,target],axis=1)

In [12]:
#Для деревянных моделей
tree_train=tree[:sep]
tree_test=tree[sep:]

tree_train=pd.concat([tree_train,target],axis=1)

In [13]:
#Для xgb
xgb_train=xgb_data[:sep]
xgb_test=xgb_data[sep:]

xgb_train=pd.concat([xgb_train,target],axis=1)

In [14]:
#Для catBoost-a
cat_train=catBoost_data[:sep]
cat_test=catBoost_data[sep:]

cat_train=pd.concat([cat_train,target],axis=1)

______

In [15]:
X_train_lin, X_test_lin, y_train_lin, y_test_lin=train_test_split(linear_train.drop("target",axis=1),
                                                                  linear_train["target"],
                                                                  test_size=0.3,
                                                                  random_state=17,
                                                                  stratify=linear_train["target"])

In [16]:
X_train_tree, X_test_tree, y_train_tree, y_test_tree=train_test_split(tree_train.drop("target",axis=1),
                                                                      tree_train["target"],
                                                                      test_size=0.3,
                                                                      random_state=17,
                                                                      stratify=tree_train["target"])

In [17]:
X_train_xgb, X_test_xgb, y_train_xgb, y_test_xgb=train_test_split(xgb_train.drop("target",axis=1),
                                                                      xgb_train["target"],
                                                                      test_size=0.3,
                                                                      random_state=17,
                                                                      stratify=xgb_train["target"])

In [18]:
X_train_cat, X_test_cat, y_train_cat, y_test_cat=train_test_split(cat_train.drop("target",axis=1),
                                                                      cat_train["target"],
                                                                      test_size=0.3,
                                                                      random_state=17,
                                                                      stratify=cat_train["target"])

##  Построение моделей

In [19]:
rf=RandomForestClassifier(random_state=17,n_jobs=-1)

lr=LogisticRegression(random_state=17,n_jobs=-1)

svmLin=LinearSVC(random_state=17)

xgb=XGBClassifier(seed=17,nthread=-1)

knn=KNeighborsClassifier(n_jobs=-1)

catCoostC=CatBoostClassifier(verbose=200,random_seed=17,iterations=400)

#### Random Forest

In [32]:
#Random Forest
rf.fit(X_train_tree,y_train_tree)
roc_auc_score(y_test_tree,rf.predict_proba(X_test_tree)[:,1])

0.67376508250602496

#### XGB

In [33]:
#XGB на tree data
xgb.fit(X_train_tree,y_train_tree)
roc_auc_score(y_test_tree,xgb.predict_proba(X_test_tree)[:,1])

0.73379398876043078

In [23]:
#Сабмит
tree_final(xgb,"xgb")

In [35]:
#XGB на данных специально подготовленные для него
xgb.fit(X_train_xgb,y_train_xgb)
roc_auc_score(y_test_xgb,xgb.predict_proba(X_test_xgb)[:,1])

0.73413993564137603

In [175]:
#Сабмит (на данных специально подготовленные для него) на лидерборже хуже предидущего
xgb_final(xgb,"xgb_specialData")

#### log reg

In [ ]:
#Обучение log reg на отмаштабированных linear данный
scaler=StandardScaler()
lr.fit(scaler.fit_transform(X_train_lin),y_train_lin)
roc_auc_score(y_test_lin,lr.predict_proba(scaler.transform(X_test_lin))[:,1])

In [ ]:
#Обучение log reg на  linear данный
lr.fit(X_train_lin,y_train_lin)
roc_auc_score(y_test_lin,lr.predict_proba(X_test_lin)[:,1])

#### KNN

In [56]:
#KNN tree data
knn.fit(X_train_tree,y_train_tree)
roc_auc_score(y_test_tree,knn.predict_proba(X_test_tree)[:,1])

0.5889554663815757

In [57]:
#KNN на linear data
knn.fit(X_train_lin,y_train_lin)
roc_auc_score(y_test_lin,knn.predict_proba(X_test_lin)[:,1])

0.58288476736413675

#### SVM

In [53]:
#svm с линейным ядром на отмаштабированных linear данный
scaler=StandardScaler()
svmLin.fit(scaler.fit_transform(X_train_lin),y_train_lin)
roc_auc_score(y_test_lin,svmLin.predict(scaler.transform(X_test_lin)))

0.6327874052997785

#### CatBoost

In [18]:
#cat boost с дефлотными настройками данные tree
catCoostC.fit(X_train_tree,y_train_tree)
roc_auc_score(y_test_tree,catCoostC.predict_proba(X_test_tree)[:,1])

0:	learn: 0.6586420	total: 247ms	remaining: 4m 6s
200:	learn: 0.2403916	total: 36.2s	remaining: 2m 23s
400:	learn: 0.2377964	total: 1m 13s	remaining: 1m 49s
600:	learn: 0.2355866	total: 1m 50s	remaining: 1m 13s
800:	learn: 0.2335584	total: 2m 29s	remaining: 37.1s


0.73911519033623996

In [82]:
#Сабмит
tree_final(catCoostC,"catboost_treeData")

0:	learn: 0.6584899	total: 270ms	remaining: 6m 17s
200:	learn: 0.2406499	total: 55.9s	remaining: 5m 33s
400:	learn: 0.2383724	total: 1m 52s	remaining: 4m 39s
600:	learn: 0.2366794	total: 2m 49s	remaining: 3m 45s
800:	learn: 0.2352162	total: 3m 49s	remaining: 2m 51s
1000:	learn: 0.2337924	total: 4m 47s	remaining: 1m 54s
1200:	learn: 0.2324842	total: 5m 42s	remaining: 56.7s


In [52]:
#cat boost с дефлотными настройками данные xgb
catCoostC.fit(X_train_xgb,y_train_xgb)
roc_auc_score(y_test_xgb,catCoostC.predict_proba(X_test_xgb)[:,1])

0:	learn: 0.6585774	total: 185ms	remaining: 3m 4s
200:	learn: 0.2407775	total: 37.8s	remaining: 2m 30s
400:	learn: 0.2380522	total: 1m 15s	remaining: 1m 52s
600:	learn: 0.2360226	total: 1m 52s	remaining: 1m 14s
800:	learn: 0.2339915	total: 2m 33s	remaining: 38s


0.74477156327247596

In [19]:
#cat boost с дефлотными настройками данные специально для него
catCoostC.fit(X_train_cat,y_train_cat,cat_features=[0,1,3,5,6,7,12,17,20,21,22,23,25])
roc_auc_score(y_test_cat,catCoostC.predict_proba(X_test_cat)[:,1])

0:	learn: 0.6593305	total: 164ms	remaining: 2m 43s
200:	learn: 0.2423677	total: 33.5s	remaining: 2m 12s
400:	learn: 0.2395806	total: 1m 5s	remaining: 1m 37s
600:	learn: 0.2376743	total: 1m 37s	remaining: 1m 4s
800:	learn: 0.2358057	total: 2m 9s	remaining: 32.2s


0.73759183176483989

In [45]:
#Сабмит
cat_final(catCoostC,"cat_boost_catdata2",[0,1,3,5,6,7,12,17,20,21,22,23,25])

0:	learn: 0.6592394	total: 243ms	remaining: 4m 2s
200:	learn: 0.2420979	total: 50.6s	remaining: 3m 21s
400:	learn: 0.2400586	total: 1m 42s	remaining: 2m 33s
600:	learn: 0.2386501	total: 2m 29s	remaining: 1m 39s
800:	learn: 0.2371027	total: 3m 18s	remaining: 49.3s
